## Import The Earth Engine Librariy & Intialize

In [1]:
import ee
import numpy as np

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Import the necessary satellite images and save it in variables

In [2]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

# Import the Landsat 8 collection for NDVI.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')

# Import the MERIT Hydro: global hydrography datasets for Flow Accumulation
fd = ee.Image("MERIT/Hydro/v1_0_1")

## Longitude & Latitude

In [3]:
u_lon = 92.12652813066406 #92.1999992
u_lat = 22.942258625256922 #22.6333308
u_poi = ee.Geometry.Point(u_lon, u_lat)
scale = 1000  # scale in meters

## Elevation

In [4]:
elv_urban_point = elv.sample(u_poi, scale).first().get('elevation').getInfo()
print('Ground elevation at urban point:', elv_urban_point, 'm')

Ground elevation at urban point: 213 m


## Land Cover

In [5]:
lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
print('Land cover value at urban point is:', lc_urban_point)

Land cover value at urban point is: 8


## Aspect

In [6]:
aspect = ee.Terrain.aspect(elv)
aspect_urban_point = aspect.sample(u_poi, scale).first().get('aspect').getInfo()
print(aspect_urban_point)

140.46868896484375


## Slope

In [7]:
slope = ee.Terrain.slope(elv)
slope_urban_point = slope.sample(u_poi, scale).first().get('slope').getInfo()
print(slope_urban_point)

0.9393998980522156


## NDVI

In [8]:
img = ee.Image(l8.filterBounds(u_poi).filterDate('2021-01-01', '2021-01-31').sort('CLOUD_COVER').first())

def meanNDVICollection(img):
    nir = img.select('B5');
    red = img.select('B4');
    ndviImage = nir.subtract(red).divide(nir.add(red)).rename('NDVI');

  # Compute the mean of NDVI over the 'region'
    ndviValue = ndviImage.reduceRegion(**{
    'geometry': img.geometry(),
    'reducer': ee.Reducer.mean(),
     'scale': 30,
    'maxPixels': 1e9
  }).get('NDVI');  # result of reduceRegion is always a dictionary, so get the element we want

    newFeature = ee.Feature(None, {
      # Adding computed NDVI value
      'NDVI': ndviValue
  }).copyProperties(img, [
      # Picking properties from original image
      'system:time_start',
      'SUN_ELEVATION'
  ])

    return newFeature

ndvi = meanNDVICollection(img)

ndvi = ndvi.get('NDVI').getInfo()
print(ndvi)

0.5232353111417488


## Flow Accumulation

In [9]:
fa = fd.select('upa')
fa_urban_point = fa.sample(u_poi, scale).getInfo()
fac = fa_urban_point['features'][0]['properties']['upa']
print(fac)

0.023036304861307144


## TWI

In [10]:
twi = np.log(fac*1000/np.tan(slope_urban_point))
print(twi)

2.824079347564467


## SPI

In [11]:
spi = fac*np.tan(slope_urban_point)
print(spi)

0.03150239390095102
